In [1]:
import pandas as pd

In [2]:
test_df = pd.read_csv('/Users/alexis/Cranfield/AI/assignment/data/test.csv')
test_df.head()

,image_id,class_name,rad_id,x_min,y_min,x_max,y_max,class_id
0,27b822c5d3b354f096dfb788fd3fa636,Nodule/Mass,R10,277.0,602.0,1177.0,2417.0,8
1,d2ca714f8f06979acfbc0992d65fb211,No finding,R3,NaN,NaN,NaN,NaN,14
2,19367e88f4634c2652d3eed0a2f1f8b2,No finding,R15,NaN,NaN,NaN,NaN,14
3,852ec6a0bcb9608ffee9b3fda0867f91,Aortic enlargement,R8,1669.0,896.0,1972.0,1326.0,0
4,aa9c3097e6d2df7216be1a1642f683c9,No finding,R5,NaN,NaN,NaN,NaN,14


In [3]:
test_pred = pd.read_csv('/Users/alexis/Cranfield/AI/assignment/Faster-R-CNN/test_predictions.csv')
test_pred.head()

,image_id,PredictionString
0,27b822c5d3b354f096dfb788fd3fa636,0 0.9913 302.8345031738281 256.54827880859375 ...
1,d2ca714f8f06979acfbc0992d65fb211,9 0.7906 294.4393005371094 200.20355224609375 ...
2,19367e88f4634c2652d3eed0a2f1f8b2,14 1.0 0 0 1 1
3,852ec6a0bcb9608ffee9b3fda0867f91,0 0.9903 322.49169921875 266.1020202636719 435...
4,aa9c3097e6d2df7216be1a1642f683c9,14 1.0 0 0 1 1


In [4]:
metadata_df = pd.read_csv('/Users/alexis/Cranfield/AI/assignment/dicom_metadata.csv')
metadata_df.head()

,File Name,Transfer Syntax UID,SOP Class UID,SOP Instance UID,Version,Photometric Interpretation,Pixel Representation,High Bit,Samples per Pixel,Bits Allocated,...,Lossy Image Compression,Lossy Image Compression Ratio,Patient's Sex,Patient's Age,Rows,Columns,Pixel Spacing,Bits Stored,Window Center,Window Width
0,f7f7736401b22c94a9d2b09425adeab9.dicom,1.2.840.10008.1.2.4.90,1.2.840.10008.5.1.4.1.1.1.1,f7f7736401b22c94a9d2b09425adeab9,INF_3.9,MONOCHROME2,0,11,1,16,...,0.0,NaN,F,037Y,3028,2517,"[0.139, 0.139]",12,2048.0,4096.0
1,93ae57ca1df1d19c6d64155e74e14d23.dicom,1.2.840.10008.1.2,1.2.840.10008.5.1.4.1.1.1.1,93ae57ca1df1d19c6d64155e74e14d23,NaN,MONOCHROME2,0,13,1,16,...,NaN,NaN,O,Y,3072,3072,"[0.139000, 0.139000]",14,10287.0,5292.0
2,f1a45afaee0efd07fef17057f3942464.dicom,1.2.840.10008.1.2.4.90,1.2.840.10008.5.1.4.1.1.1,f1a45afaee0efd07fef17057f3942464,OFFIS_DCMTK_360,MONOCHROME1,0,11,1,16,...,0.0,NaN,M,NaN,2880,2304,"[0.15, 0.15]",12,2047.0,4095.0
3,559fdfb3b9db3eb206b9521824f716e3.dicom,1.2.840.10008.1.2,1.2.840.10008.5.1.4.1.1.1,559fdfb3b9db3eb206b9521824f716e3,OFFIS_DCMTK_360,MONOCHROME2,0,13,1,16,...,NaN,NaN,M,038Y,3072,2540,"[0.140, 0.140]",14,10075.0,10242.0
4,371d6f540ac601affe81e547b05c551a.dicom,1.2.840.10008.1.2,1.2.840.10008.5.1.4.1.1.1,371d6f540ac601affe81e547b05c551a,OFFIS_DCMTK_360,MONOCHROME2,0,13,1,16,...,NaN,NaN,M,060Y,3072,3072,"[0.140, 0.140]",14,8755.0,10228.0


In [5]:
image_sizes_df = metadata_df[['File Name', 'Rows', 'Columns']]
image_sizes_df.head()

,File Name,Rows,Columns
0,f7f7736401b22c94a9d2b09425adeab9.dicom,3028,2517
1,93ae57ca1df1d19c6d64155e74e14d23.dicom,3072,3072
2,f1a45afaee0efd07fef17057f3942464.dicom,2880,2304
3,559fdfb3b9db3eb206b9521824f716e3.dicom,3072,2540
4,371d6f540ac601affe81e547b05c551a.dicom,3072,3072


In [6]:
# """ import logging
# import h5py

# logging.basicConfig(
#     level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
# )

# def get_pixel_array_from_hdf5(hdf5_path, filename):
#     """
#     Retrieves the pixel array for a given filename from an HDF5 file.

#     Parameters:
#     - hdf5_path: Path to the HDF5 file.
#     - filename: The original file name of the DICOM image.

#     Returns:
#     - A numpy array of the pixel data if found; None otherwise.
#     """
#     with h5py.File(hdf5_path, "r") as hdf5_file:
#         # Attempt to access the dataset directly by filename.
#         # Adjust this part if a more complex naming convention is used.
#         unique_filename = None
#         for key in hdf5_file.keys():
#             if filename in key:
#                 unique_filename = key
#                 break
                
#         if unique_filename is not None:
#             pixel_array = hdf5_file[unique_filename][:]
#             return pixel_array
#         else:
#             logging.error(f"File {filename} not found in HDF5.")
#             return None """

In [7]:
# import matplotlib.pyplot as plt

# filename = "f7f7736401b22c94a9d2b09425adeab9"  # This should be the base name of the file you're looking for.
# pixel_array = get_pixel_array_from_hdf5("/Volumes/ALEXIS/ai_project_cranfield/dicom_images_final.hdf5", filename+".dicom")
# plt.imshow(pixel_array, cmap="gray")

# # print the size of the image
# original_size = pixel_array.shape[:2]  # (hauteur, largeur)
# print(original_size)

In [8]:
test_prediction_reformatted = pd.read_csv("/Users/alexis/Cranfield/AI/assignment/Faster-R-CNN/test_predictions_reformatted.csv")
test_prediction_reformatted

,image_id,class_id,score,x_min,y_min,x_max,y_max
0,27b822c5d3b354f096dfb788fd3fa636,0,0.9913,302.834503,256.548279,400.121002,359.355103
1,27b822c5d3b354f096dfb788fd3fa636,11,0.9786,280.547729,148.544403,357.263306,173.076050
2,27b822c5d3b354f096dfb788fd3fa636,8,0.9731,481.073578,412.827332,553.028870,479.975830
3,27b822c5d3b354f096dfb788fd3fa636,8,0.9301,108.356094,585.596802,126.756836,599.684204
4,27b822c5d3b354f096dfb788fd3fa636,8,0.9037,151.903030,518.127075,176.650681,540.022095
...,...,...,...,...,...,...,...
57737,eb6c714df22142229464c6b83e47d7d6,5,0.1241,70.623360,157.016769,323.330566,662.195984
57738,eb6c714df22142229464c6b83e47d7d6,13,0.1151,510.774078,171.422897,540.845215,248.329208
57739,eb6c714df22142229464c6b83e47d7d6,5,0.1090,106.454468,239.314194,281.668915,508.423096
57740,eb6c714df22142229464c6b83e47d7d6,11,0.1039,208.922638,155.983871,315.305450,189.295242


In [9]:
test_df['x_min'] = test_df['x_min'].fillna(0)
test_df['y_min'] = test_df['y_min'].fillna(0)
test_df['x_max'] = test_df['x_max'].fillna(1)
test_df['y_max'] = test_df['y_max'].fillna(1)

In [10]:
test_df

,image_id,class_name,rad_id,x_min,y_min,x_max,y_max,class_id
0,27b822c5d3b354f096dfb788fd3fa636,Nodule/Mass,R10,277.0,602.0,1177.0,2417.0,8
1,d2ca714f8f06979acfbc0992d65fb211,No finding,R3,0.0,0.0,1.0,1.0,14
2,19367e88f4634c2652d3eed0a2f1f8b2,No finding,R15,0.0,0.0,1.0,1.0,14
3,852ec6a0bcb9608ffee9b3fda0867f91,Aortic enlargement,R8,1669.0,896.0,1972.0,1326.0,0
4,aa9c3097e6d2df7216be1a1642f683c9,No finding,R5,0.0,0.0,1.0,1.0,14
...,...,...,...,...,...,...,...,...
10183,54e6184c63c75a9695d7effc17969ad0,Cardiomegaly,R8,731.0,1431.0,1697.0,1842.0,3
10184,7237fe007c5cab239011e89137eee3a7,Other lesion,R17,667.0,752.0,1282.0,1777.0,9
10185,3b2087f26637d64244ec11da41770aad,Aortic enlargement,R9,1604.0,705.0,1850.0,931.0,0
10186,eb6c714df22142229464c6b83e47d7d6,Lung Opacity,R10,494.0,1293.0,866.0,1632.0,7


In [11]:
image_sizes_df = image_sizes_df.rename(columns={"File Name": "image_id"})
# Drop .dicom from the image_id
image_sizes_df["image_id"] = image_sizes_df["image_id"].str.replace(".dicom", "")
image_sizes_df

,image_id,Rows,Columns
0,f7f7736401b22c94a9d2b09425adeab9,3028,2517
1,93ae57ca1df1d19c6d64155e74e14d23,3072,3072
2,f1a45afaee0efd07fef17057f3942464,2880,2304
3,559fdfb3b9db3eb206b9521824f716e3,3072,2540
4,371d6f540ac601affe81e547b05c551a,3072,3072
...,...,...,...
14995,0c5ff01c7bfb4362fcd98f36e555b08c,2880,2304
14996,3286115baacb6ed3affc5c46c9205b7a,2880,2304
14997,d44766bf0eb902fbc1848f418417c45f,2208,2104
14998,cd7b83d24ee9c092a0f32cb020da6dac,2926,2847


In [12]:
test_prediction_reformatted = test_prediction_reformatted.merge(image_sizes_df, on="image_id", how="left")
test_prediction_reformatted

,image_id,class_id,score,x_min,y_min,x_max,y_max,Rows,Columns
0,27b822c5d3b354f096dfb788fd3fa636,0,0.9913,302.834503,256.548279,400.121002,359.355103,2968,2580
1,27b822c5d3b354f096dfb788fd3fa636,11,0.9786,280.547729,148.544403,357.263306,173.076050,2968,2580
2,27b822c5d3b354f096dfb788fd3fa636,8,0.9731,481.073578,412.827332,553.028870,479.975830,2968,2580
3,27b822c5d3b354f096dfb788fd3fa636,8,0.9301,108.356094,585.596802,126.756836,599.684204,2968,2580
4,27b822c5d3b354f096dfb788fd3fa636,8,0.9037,151.903030,518.127075,176.650681,540.022095,2968,2580
...,...,...,...,...,...,...,...,...,...
57737,eb6c714df22142229464c6b83e47d7d6,5,0.1241,70.623360,157.016769,323.330566,662.195984,3073,2532
57738,eb6c714df22142229464c6b83e47d7d6,13,0.1151,510.774078,171.422897,540.845215,248.329208,3073,2532
57739,eb6c714df22142229464c6b83e47d7d6,5,0.1090,106.454468,239.314194,281.668915,508.423096,3073,2532
57740,eb6c714df22142229464c6b83e47d7d6,11,0.1039,208.922638,155.983871,315.305450,189.295242,3073,2532


In [13]:
test_prediction_reformatted = test_prediction_reformatted.rename(columns={"Rows": "image_height", "Columns": "image_width"})

In [14]:

def resize_image(img_arr, bboxes, h, w):
    """
    :param img_arr: original image as a numpy array
    :param bboxes: bboxes as numpy array where each row is 'x_min', 'y_min', 'x_max', 'y_max', "class_id"
    :param h: resized height dimension of image
    :param w: resized weight dimension of image
    :return: dictionary containing {image:transformed, bboxes:['x_min', 'y_min', 'x_max', 'y_max', "class_id"]}
    """
    # create resize transform pipeline
    transform = albumentations.Compose(
        [albumentations.Resize(height=h, width=w, always_apply=True)],
        bbox_params=albumentations.BboxParams(format='pascal_voc'))

    transformed = transform(image=img_arr, bboxes=bboxes)

    return transformed

In [15]:
def resizeBox(x, x_target, x_original):
    scale = x_original / x_target
    return x * scale

target_size = (800, 800)

# Resize the bounding boxes (only when class_id is not 14
test_prediction_reformatted["x_min"] = test_prediction_reformatted.apply(lambda x: resizeBox(x.x_min, target_size[0], x.image_width) if x.class_id != 14 else x.x_min, axis=1)
test_prediction_reformatted["x_max"] = test_prediction_reformatted.apply(lambda x: resizeBox(x.x_max, target_size[0], x.image_width) if x.class_id != 14 else x.x_max, axis=1)
test_prediction_reformatted["y_min"] = test_prediction_reformatted.apply(lambda x: resizeBox(x.y_min, target_size[1], x.image_height) if x.class_id != 14 else x.y_min, axis=1)
test_prediction_reformatted["y_max"] = test_prediction_reformatted.apply(lambda x: resizeBox(x.y_max, target_size[1], x.image_height) if x.class_id != 14 else x.y_max, axis=1)

In [16]:
test_df

,image_id,class_name,rad_id,x_min,y_min,x_max,y_max,class_id
0,27b822c5d3b354f096dfb788fd3fa636,Nodule/Mass,R10,277.0,602.0,1177.0,2417.0,8
1,d2ca714f8f06979acfbc0992d65fb211,No finding,R3,0.0,0.0,1.0,1.0,14
2,19367e88f4634c2652d3eed0a2f1f8b2,No finding,R15,0.0,0.0,1.0,1.0,14
3,852ec6a0bcb9608ffee9b3fda0867f91,Aortic enlargement,R8,1669.0,896.0,1972.0,1326.0,0
4,aa9c3097e6d2df7216be1a1642f683c9,No finding,R5,0.0,0.0,1.0,1.0,14
...,...,...,...,...,...,...,...,...
10183,54e6184c63c75a9695d7effc17969ad0,Cardiomegaly,R8,731.0,1431.0,1697.0,1842.0,3
10184,7237fe007c5cab239011e89137eee3a7,Other lesion,R17,667.0,752.0,1282.0,1777.0,9
10185,3b2087f26637d64244ec11da41770aad,Aortic enlargement,R9,1604.0,705.0,1850.0,931.0,0
10186,eb6c714df22142229464c6b83e47d7d6,Lung Opacity,R10,494.0,1293.0,866.0,1632.0,7


In [17]:
# Number of unique image_ids in the test set
len(test_df["image_id"].unique())

7028

In [18]:
# Number of unique image_ids in the prediction set
len(test_prediction_reformatted["image_id"].unique())

7028

In [19]:
numberOfRecordsPerClass = test_prediction_reformatted.groupby("class_id").size().reset_index(name="counts")
numberOfRecordsPerClass

,class_id,counts
0,0,3843
1,1,407
2,2,1584
3,3,3024
4,4,768
5,5,3902
6,6,2234
7,7,5119
8,8,3560
9,9,4018


In [20]:
numberOfRecordsPerClass["counts"].sum()

57742

In [21]:
numberOfRecordsPerClassTarget = test_df.groupby("class_id").size().reset_index(name="counts")
numberOfRecordsPerClassTarget

,class_id,counts
0,0,1074
1,1,42
2,2,144
3,3,814
4,4,83
5,5,150
6,6,187
7,7,373
8,8,387
9,9,331


In [22]:
numberOfRecordsPerClassTarget["counts"].sum()

10188

In [40]:
import joblib

pipeline = joblib.load("/Users/alexis/Cranfield/AI/assignment/random_forest.sav")

# Make predictions
test_prediction_reformatted["confidence"] = pipeline.predict(test_prediction_reformatted[["x_min", "y_min", "x_max", "y_max"]])

In [23]:
def calculate_iou(boxA, boxB):
    """Calcule l'Intersection over Union (IoU) entre deux boîtes."""
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

In [24]:
def evaluate_predictions(ground_truths, predictions, iou_threshold=0.4):
    TP = 0
    FP = 0
    FN = 0

    # Enregistre si une vérité terrain a été appariée à une prédiction
    matched_gt = {gt_index: False for gt_index, _ in enumerate(ground_truths)}

    for pred in predictions:
        best_iou = 0
        best_gt_index = None
        for gt_index, gt in enumerate(ground_truths):
            if (
                gt["image_id"] == pred["image_id"]
                and gt["class_id"] == pred["class_id"]
            ):
                iou = calculate_iou(
                    [pred["x_min"], pred["y_min"], pred["x_max"], pred["y_max"]],
                    [gt["x_min"], gt["y_min"], gt["x_max"], gt["y_max"]],
                )
                if iou > best_iou:
                    best_iou = iou
                    best_gt_index = gt_index

        if best_iou > iou_threshold:
            TP += 1
            if best_gt_index is not None:
                matched_gt[best_gt_index] = True
        else:
            FP += 1

    # Les vérités terrain non appariées sont des faux négatifs
    FN = sum(not matched for matched in matched_gt.values())

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score = (
        2 * (precision * recall) / (precision + recall)
        if (precision + recall) > 0
        else 0
    )

    return {
        "TP": TP,
        "FP": FP,
        "FN": FN,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1_score,
    }

In [25]:
def evaluate_predictions_per_class(ground_truths, predictions, iou_threshold=0.4, no_finding_class_id=14):
    class_metrics = {}

    # Initialiser les compteurs pour toutes les classes dans les vérités terrain
    for gt in ground_truths:
        class_id = gt["class_id"]
        if class_id not in class_metrics:
            class_metrics[class_id] = {"TP": 0, "FP": 0, "FN": 0, "total_gt": 0, "TN": 0}
        class_metrics[class_id]["total_gt"] += 1

    # Traiter les prédictions
    for pred in predictions:
        class_id = pred["class_id"]
        if class_id not in class_metrics:
            class_metrics[class_id] = {"TP": 0, "FP": 0, "FN": 0, "total_gt": 0, "TN": 0}

        best_iou = 0
        matched_gt = None
        for gt in ground_truths:
            if gt["class_id"] == class_id and gt["image_id"] == pred["image_id"]:
                iou = calculate_iou(
                    [pred["x_min"], pred["y_min"], pred["x_max"], pred["y_max"]],
                    [gt["x_min"], gt["y_min"], gt["x_max"], gt["y_max"]]
                )
                if iou > best_iou:
                    best_iou = iou
                    matched_gt = gt

        if best_iou >= iou_threshold:
            class_metrics[class_id]["TP"] += 1
            # Marquer la vérité terrain comme appariée
            if matched_gt:
                ground_truths.remove(matched_gt)
        else:
            class_metrics[class_id]["FP"] += 1

    # Calculer l'accuracy pour chaque classe, incluant "No finding"
    for class_id, metrics in class_metrics.items():
        TP, FP, FN = metrics["TP"], metrics["FP"], metrics["FN"]
        metrics["Accuracy"] = TP / (TP + FP + FN) if (TP + FP + FN) > 0 else 0
        
    return class_metrics


In [26]:
def evaluate_no_finding(ground_truths, predictions, no_finding_class_id=14):
    # Initialiser les compteurs
    TP = 0  # Correctement prédit comme "No finding"
    FP = 0  # Incorrectement prédit comme "No finding" ou prédit quelque chose d'autre quand c'était "No finding"
    FN = 0  # Manqué de prédire "No finding"

    # Ensemble des IDs des images "No finding"
    no_finding_image_ids = {gt['image_id'] for gt in ground_truths if gt['class_id'] == no_finding_class_id}

    # Ensemble des IDs des images pour lesquelles des prédictions ont été faites
    predicted_image_ids = {pred['image_id'] for pred in predictions}

    # Compter TP et FP
    for pred in predictions:
        if pred['class_id'] == no_finding_class_id:
            if pred['image_id'] in no_finding_image_ids:
                TP += 1  # Prédiction correcte de "No finding"
            else:
                FP += 1  # Prédiction incorrecte comme "No finding"

    # Compter FN - "No finding" manqué
    FN = len(no_finding_image_ids) - TP

    accuracy = (TP) / (TP + FP + FN) if (TP + FP + FN) > 0 else 0

    return {'TP': TP, 'FP': FP, 'FN': FN, 'Accuracy': accuracy}


In [27]:
ground_truths = test_df.to_dict('records')
predictions = test_prediction_reformatted.to_dict('records')

evaluate_no_finding(ground_truths, predictions)

{'TP': 2594, 'FP': 0, 'FN': 1473, 'Accuracy': 0.6378165724120973}

In [36]:
ground_truths = test_df.to_dict('records')
predictions = test_prediction_reformatted.to_dict('records')

confidence_threshold = 0.1
predictions = [pred for pred in predictions if pred["score"] > confidence_threshold]
# For each image, if there is no prediction for "No finding", add a prediction with class_id 14 and confidence 1
image_ids = set([pred["image_id"] for pred in predictions])
for image_id in image_ids:
    no_finding_predictions = [pred for pred in predictions if pred["image_id"] == image_id and pred["class_id"] == 14]
    if len(no_finding_predictions) == 0:
        predictions.append({
            "image_id": image_id,
            "x_min": 0,
            "y_min": 0,
            "x_max": 1,
            "y_max": 1,
            "class_id": 14,
            "score": 1
        })

In [37]:
from collections import defaultdict
import torch

# Initialize dictionaries to hold aggregated data
aggregated_preds = defaultdict(lambda: {'boxes': [], 'scores': [], 'labels': []})
aggregated_gts = defaultdict(lambda: {'boxes': [], 'labels': []})

# Aggregate predictions by image
for pred in predictions:
    aggregated_preds[pred['image_id']]['boxes'].append([pred['x_min'], pred['y_min'], pred['x_max'], pred['y_max']])
    aggregated_preds[pred['image_id']]['scores'].append(pred['score'])
    aggregated_preds[pred['image_id']]['labels'].append(pred['class_id'])

# Aggregate ground truths by image
for gt in ground_truths:
    aggregated_gts[gt['image_id']]['boxes'].append([gt['x_min'], gt['y_min'], gt['x_max'], gt['y_max']])
    aggregated_gts[gt['image_id']]['labels'].append(gt['class_id'])

# Convert aggregated data into the required format
preds_formatted = [{
    'boxes': torch.tensor(aggregated_preds[image_id]['boxes']),
    'scores': torch.tensor(aggregated_preds[image_id]['scores']),
    'labels': torch.tensor(aggregated_preds[image_id]['labels'])
} for image_id in aggregated_preds]

gt_formatted = [{
    'boxes': torch.tensor(aggregated_gts[image_id]['boxes']),
    'labels': torch.tensor(aggregated_gts[image_id]['labels'])
} for image_id in aggregated_gts if image_id in aggregated_preds]  # Ensure matching image_ids

# Verify the lengths match
assert len(preds_formatted) == len(gt_formatted), "Mismatch in the number of images between preds and gt."

In [38]:
from torchmetrics.detection.mean_ap import MeanAveragePrecision

# Create a MeanAveragePrecision object
mAP_metric = MeanAveragePrecision(box_format='xyxy', iou_type='bbox', class_metrics=True, iou_thresholds=[0.4])

# Update the metric with the data
mAP_metric.update(preds=preds_formatted, target=gt_formatted)

# Compute the final mAP
mAP = mAP_metric.compute()

mAP

{'map': tensor(0.0942),
 'map_50': tensor(-1.),
 'map_75': tensor(-1.),
 'map_small': tensor(0.1266),
 'map_medium': tensor(0.0337),
 'map_large': tensor(0.0668),
 'mar_1': tensor(0.3014),
 'mar_10': tensor(0.4466),
 'mar_100': tensor(0.4519),
 'mar_small': tensor(0.3079),
 'mar_medium': tensor(0.1945),
 'mar_large': tensor(0.4292),
 'map_per_class': tensor([0.1148, 0.0553, 0.0169, 0.2532, 0.0218, 0.1002, 0.0322, 0.0218, 0.0330,
         0.0285, 0.0615, 0.0287, 0.1147, 0.0369, 0.4940]),
 'mar_100_per_class': tensor([0.4926, 0.3333, 0.3264, 0.6683, 0.3253, 0.7200, 0.4278, 0.3512, 0.3152,
         0.2598, 0.4812, 0.4325, 0.3824, 0.4112, 0.8521]),
 'classes': tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
        dtype=torch.int32)}

In [39]:
for class_id in test_df["class_id"].unique():
    ground_truths_class = [gt for gt in ground_truths if gt["class_id"] == class_id]
    predictions_class = [pred for pred in predictions if pred["class_id"] == class_id]
    df_eval_class = evaluate_predictions(ground_truths_class, predictions_class, iou_threshold=0.3)
    print(f"Class {class_id}")
    print(df_eval_class)

Class 8
{'TP': 151, 'FP': 3409, 'FN': 263, 'Precision': 0.04241573033707865, 'Recall': 0.3647342995169082, 'F1 Score': 0.07599396074484147}
Class 14
{'TP': 4066, 'FP': 2961, 'FN': 707, 'Precision': 0.5786253024050092, 'Recall': 0.8518751309448984, 'F1 Score': 0.6891525423728813}
Class 0
{'TP': 691, 'FP': 3151, 'FN': 514, 'Precision': 0.17985424258198854, 'Recall': 0.5734439834024896, 'F1 Score': 0.2738260352684763}
Class 3
{'TP': 710, 'FP': 2313, 'FN': 169, 'Precision': 0.23486602712537213, 'Recall': 0.8077360637087599, 'F1 Score': 0.3639159405433111}
Class 6
{'TP': 134, 'FP': 2097, 'FN': 92, 'Precision': 0.0600627521290901, 'Recall': 0.5929203539823009, 'F1 Score': 0.1090761090761091}
Class 11
{'TP': 743, 'FP': 11485, 'FN': 367, 'Precision': 0.06076218514883873, 'Recall': 0.6693693693693694, 'F1 Score': 0.11141100614784825}
Class 4
{'TP': 42, 'FP': 724, 'FN': 55, 'Precision': 0.05483028720626632, 'Recall': 0.4329896907216495, 'F1 Score': 0.09733487833140209}
Class 13
{'TP': 648, 'FP':